In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('df_for_modeling.csv')

In [4]:
df.columns.values

array(['id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'int_rate',
       'installment', 'grade', 'sub_grade', 'emp_length', 'annual_inc',
       'dti', 'delinq_2yrs', 'fico_range_low', 'fico_range_high',
       'inq_last_6mths', 'mths_since_last_delinq',
       'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal',
       'revol_util', 'total_acc', 'out_prncp', 'out_prncp_inv',
       'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp',
       'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'last_pymnt_amnt',
       'last_fico_range_high', 'last_fico_range_low',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'policy_code', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal',
       'open_acc_6m', 'open_act_il', 'open_il_12m', 'open_il_24m',
       'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m',
       'open_rv_24m', 'max_bal_bc', 'all_util', 'total_rev_hi_lim',
       'inq_fi', 'total_cu_tl', 'i

In [5]:
#Logistic regression is a distance based algorithm, therefore we need to standardize the data.
feat_no_stan = ['y', 'issued_mark', 'ANY', 'MORTGAGE', 'OWN', 'RENT', 'Not Verified', 'Source Verified',
                'f', 'w', 'Individual', 'Joint App']

In [6]:
df.drop('id', axis = 1, inplace = True)

In [7]:
from sklearn import preprocessing

In [13]:
for feat in df.columns.values:
    if feat not in feat_no_stan:
        df[feat] = preprocessing.scale(df[feat])

In [14]:
df['loan_amnt'].head()

0   -0.005538
1   -0.449807
2   -0.566720
3    0.836233
4   -0.800545
Name: loan_amnt, dtype: float64

In [29]:
df.issued_mark.value_counts()

0    235720
1     53118
Name: issued_mark, dtype: int64

In [30]:
df.shape

(288838, 114)

In [15]:
#Now splitting the training set and the validation set.
train = df[df['issued_mark'] == 0]
test = df[df['issued_mark'] == 1]

In [31]:
train.shape

(235720, 113)

In [32]:
test.shape

(53118, 113)

In [16]:
train.drop('issued_mark', axis = 1, inplace = True)
test.drop('issued_mark', axis = 1, inplace = True)

/Users/haoli/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [17]:
#The target value is imbalanced.
train.y.value_counts()

0    197066
1     38654
Name: y, dtype: int64

In [18]:
train_y_0 = train[train['y'] == 0]
train_y_1 = train[train['y'] == 1]

In [19]:
train_sample = train_y_0.sample(38654)

In [20]:
train_new = pd.concat([train_sample, train_y_1])

In [33]:
train_new.y.value_counts()

1    38654
0    38654
Name: y, dtype: int64

In [21]:
train_x = train_new.drop('y', axis = 1)

In [35]:
train_x.shape

(77308, 112)

In [22]:
train_y = train_new['y']

In [36]:
train_y.shape

(77308,)

In [23]:
test_x = test.drop('y', axis = 1)
test_y = test['y']

In [37]:
test_x.shape

(53118, 112)

In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [26]:
lr = LogisticRegression()

In [38]:
lr.fit(train_x, train_y)

/Users/haoli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [39]:
print(metrics.classification_report(test_y, lr.predict(test_x)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     42154
           1       1.00      1.00      1.00     10964

    accuracy                           1.00     53118
   macro avg       1.00      1.00      1.00     53118
weighted avg       1.00      1.00      1.00     53118



In [40]:
lr.predict(test_x)

array([0, 0, 0, ..., 1, 0, 0])

In [41]:
test_y

235720    0
235721    0
235722    0
235723    0
235724    0
         ..
288833    0
288834    0
288835    1
288836    0
288837    0
Name: y, Length: 53118, dtype: int64

In [42]:
train_x = train.drop('y', axis = 1)
train_y = train['y']

In [43]:
lr.fit(train_x, train_y)

/Users/haoli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [44]:
print(metrics.classification_report(test_y, lr.predict(test_x)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     42154
           1       1.00      0.99      1.00     10964

    accuracy                           1.00     53118
   macro avg       1.00      1.00      1.00     53118
weighted avg       1.00      1.00      1.00     53118

